In [1]:
import ctypes
import os
import subprocess

import pygccxml
from pygccxml import declarations, parser, utils

import os
import subprocess

__here__ = os.path.abspath(".")# Configure the xml generatortry:    gcc_path = subprocess.check_output(["where", "gcc"]).decode("UTF-8").strip()    gcc_path = os.path.abspath(gcc_path)    assert os.path.isfile(gcc_path)except subprocess.CalledProcessError:    passos.chdir(r"castxml\bin")generator_path, generator_name = utils.find_xml_generator()os.chdir(__here__)xml_generator_config = parser.xml_generator_configuration_t(    xml_generator_path=generator_path,    xml_generator=generator_name,    compiler_path=gcc_path,)

# Configure the xml generator
try:
    gcc_path = subprocess.check_output(["where", "gcc"]).decode("UTF-8").strip()
    gcc_path = os.path.abspath(gcc_path)
    assert os.path.isfile(gcc_path)
except subprocess.CalledProcessError:
    pass

os.chdir(r"castxml\bin")
generator_path, generator_name = utils.find_xml_generator()
os.chdir(__here__)

xml_generator_config = parser.xml_generator_configuration_t(
    xml_generator_path=generator_path,
    xml_generator=generator_name,
    compiler_path=gcc_path
)

In [2]:
__here__ = os.path.abspath(".")# Configure the xml generatortry:    gcc_path = subprocess.check_output(["where", "gcc"]).decode("UTF-8").strip()    gcc_path = os.path.abspath(gcc_path)    assert os.path.isfile(gcc_path)except subprocess.CalledProcessError:    passos.chdir(r"castxml\bin")generator_path, generator_name = utils.find_xml_generator()os.chdir(__here__)xml_generator_config = parser.xml_generator_configuration_t(    xml_generator_path=generator_path,    xml_generator=generator_name,    compiler_path=gcc_path,)

header = os.path.abspath("Development/CANapAPI.h")
os.path.exists(header)

with open(header, "r") as fid:
    data = fid.read()

In [3]:
base, ext = os.path.splitext(header)
header2 = os.path.abspath(f"{base}2{ext}")

In [4]:
# These are to make 
header_fix = """
// Push these out of the way
#define ASAP3_EXPORT
#define CALL_CONV
#define WINAPI

// Windows Path
#define _MAX_PATH 256
#define MAX_PATH 256
#define CALLBACK __stdcall

// Some typedefs
typedef int BOOL;
typedef unsigned long DWORD;
typedef unsigned char BYTE;
typedef unsigned int BOOLEAN;
typedef unsigned int UINT;
typedef unsigned long ULONG_PTR;
typedef ULONG_PTR DWORD_PTR;
#define NULL 0
"""

with open(header2, "w") as fid:
    fid.write(header_fix)
    fid.write(data)
header_declarations = parser.parse(files=[header2], config=xml_generator_config)
ns = header_declarations[0]

INFO Parsing source file "C:\Projects\python_CANape\Development\CANapAPI2.h" ... 


In [6]:
class ctypes_typedef_gen(object):
    def __init__(self, typedef):
        self.typedef = typedef

    @property
    def decl_str(self):
        return self.typedef.decl_type.decl_string

    @property
    def unsigned(self):
        return "unsigned" in self.decl_str

    @property
    def name(self):
        return self.typedef.name

    @property
    def short_name(self):
        return self.name.split("_")[0]

    @property
    def c_type(self):
        if self.decl_str.startswith("::"):
            return self.decl_str.strip(":")
        if hasattr(ctypes, f"c_{self.short_name}"):
            return f"c_{self.short_name}"
        if hasattr(ctypes, f"c_{self.decl_str}"):
            return f"c_{self.decl_str}"
        if self.decl_str.endswith(" int"):
            if "unsigned" in n.decl_str:
                unsigned="u"
            else:
                unsigned=""
            base = n.decl_str.split()[0]
            c_type = f"c_{unsigned}{base}"
            if hasattr(ctypes, c_type):
                return c_type
            base = n.decl_str.split()[1]
            c_type = f"c_{unsigned}{base}"
            if hasattr(ctypes, c_type):
                return c_type
        
        type_map = {
            "unsigned char": "c_ubyte",
            "void *": "c_void_p",
        }
        if self.decl_str in type_map:
            return type_map[self.decl_str]

        raise Exception(f"Unimplemented type {self.decl_str}")

    @property
    def python_line(self):
        return f"{self.name}=ctypes.{self.c_type}"

    def __repr__(self):
        return "<{}={}>".format(self.name, self.decl_str)


In [15]:
deffed = list()
for typedef in ns.typedefs():
    if typedef.name.startswith("_"):
        continue
    n = ctypes_typedef_gen(typedef)
    if n.name == n.decl_str.strip("::"):
        #Structs
        continue
    if n.name == "FNCDIAGNOFIFICATION":
        #TODO
        continue
    print(f"{n.name}={n.c_type}")
    deffed.append(n.name)
    if n.name=="DBFileInfo":
        break

BOOL=c_int
DWORD=c_ulong
BYTE=c_ubyte
BOOLEAN=c_uint
UINT=c_uint
ULONG_PTR=c_ulong
DWORD_PTR=ULONG_PTR
TAsap3DiagHdl=c_ulong
DiagNumericParamater=NumericParamater
TMeasurementListEntry=MeasurementListEntry
TAsap3Hdl=tAsap3Hdl *


Exception: Unimplemented type long unsigned int *

In [ ]:
def find_ctypes_type(type_str):
    if hasattr(ctypes, f"c_{self.short_name}"):
        return f"c_{self.short_name}"
    

In [22]:
class ctypes_typedef_gen(object):
    def __init__(self, typedef):
        self.typedef = typedef

    @property
    def decl_str(self):
        return self.typedef.decl_type.decl_string

    @property
    def unsigned(self):
        return "unsigned" in self.decl_str

    @property
    def name(self):
        return self.typedef.name

    @property
    def short_name(self):
        return self.name.split("_")[0]

    @property
    def c_type(self):
        if self.decl_str.startswith("::"):
            return self.decl_str.strip(":")
        if hasattr(ctypes, f"c_{self.short_name}"):
            return f"c_{self.short_name}"
        if hasattr(ctypes, f"c_{self.decl_str}"):
            return f"c_{self.decl_str}"
        if self.decl_str.endswith(" int"):
            if "unsigned" in n.decl_str:
                unsigned="u"
            else:
                unsigned=""
            base = n.decl_str.split()[0]
            c_type = f"c_{unsigned}{base}"
            if hasattr(ctypes, c_type):
                return c_type
            base = n.decl_str.split()[1]
            c_type = f"c_{unsigned}{base}"
            if hasattr(ctypes, c_type):
                return c_type
        if self.decl_str.endswith("*"):
            return "POINTER()"
        
        type_map = {
            "unsigned char": "c_ubyte",
            "void *": "c_void_p",
        }
        if self.decl_str in type_map:
            return type_map[self.decl_str]

        raise Exception(f"Unimplemented type {self.decl_str}")

    @property
    def python_line(self):
        return f"{self.name}=ctypes.{self.c_type}"

    def __repr__(self):
        return "<{}={}>".format(self.name, self.decl_str)


In [34]:
deffed = list()
for typedef in ns.typedefs():
    if typedef.name.startswith("_"):
        continue
    n = ctypes_typedef_gen(typedef)
    if n.name == n.name.upper():
        # Specials
        continue
    if n.name == n.decl_str.strip("::"):
        #Structs
        continue
    if n.name == "FNCDIAGNOFIFICATION":
        #TODO
        continue
    print(f"{n.name}=ctypes.{n.c_type}")
    deffed.append(n.name)
    if n.name=="DBFileInfo":
        break

TAsap3DiagHdl=ctypes.c_ulong
DiagNumericParamater=ctypes.NumericParamater
TMeasurementListEntry=ctypes.MeasurementListEntry
TAsap3Hdl=ctypes.tAsap3Hdl *
TRecorderID=ctypes.POINTER()
TModulHdl=ctypes.c_ushort
TScriptHdl=ctypes.DWORD_PTR
TTime=ctypes.c_ulong


In [36]:
def cpptype_to_ctypes(t):
    if declarations.is_array(t):
        if isinstance(t.base, pygccxml.declarations.cpptypes.char_t):
            return "ctypes.c_char_p({})".format(t.size)
    if declarations.is_pointer(t):
        if hasattr(t, 'base'):
            if declarations.is_pointer(t.base):
                if hasattr(hasattr(t, 'base'), 'base') and t.base.base.decl_string  == "char":
                    return "ctypes.POINTER(ctypes.c_char_p)"  
            if t.base.decl_string == "char":
                return "ctypes.c_char_p"
            if t.base.decl_string == "void": 
                return "ctypes.c_void_p"
        return "ctypes.POINTER({})".format(cpptype_to_ctypes(declarations.remove_pointer(t)))
    if declarations.is_const(t):
        return cpptype_to_ctypes(declarations.remove_const(t))
    if not hasattr(t, "decl_string"):
        return "!!!!"
    if t.decl_string.startswith("::"):
        return t.decl_string[2:]
    tp = t.decl_string.split(" ")
    if len(tp)==1:
        return "ctypes.c_" + tp[0]
        if tp[0] == "bool":
            return "ctypes.c_bool"
        if tp[0] == "char":
            return "ctypes.c_char"
        if tp[0] == "double":
            return "ctypes.c_double"
        if tp[0] == "int":
            return "ctypes.c_int"
    if len(tp)==2:
        if tp[1] == "volatile":
            return "ctypes.c_{}".format(tp[0])
        if tp[0] == "unsigned":
            if tp[1] == "char":
                tp[1] = "byte"
            return "ctypes.c_u{}".format(tp[1])
        if tp[1] == "int":
            return "ctypes.c_{}".format(tp[0])
    if len(tp)==3:
        if tp[1]=="unsigned":
            return "ctypes.c_u{}".format(tp[0])
    raise(Exception(t.decl_string))

In [40]:
i=1
for x in ns.classes():
    if declarations.is_struct(x): #x.name in ["Appversion", "TApplicationType", "TApplicationID", "TAsap3Hdl", "version_t"]:
        if len(x.name)==0:
            break
        print("class {}(ctypes.Structure):".format(x.name))
        print("{}_fields_ = [".format(" "*4*i))
        for var in x.variables():
            print("{}(\"{}\", {}),".format(" "*(4*(i+1)),var.name,cpptype_to_ctypes(var.decl_type)))
        print("{}]".format(" "*(4*i)))
        print()

class TApplicationID(ctypes.Structure):
    _fields_ = [
        ("tApplicationType", TApplicationType),
        ("tApplicationPath", ctypes.c_char_p(256)),
    ]

class DiagJobResponse(ctypes.Structure):
    _fields_ = [
        ("job_responsestring", ctypes.c_char_p),
        ("job_responseValue", ctypes.c_double),
    ]

class NumericParamater(ctypes.Structure):
    _fields_ = [
        ("DiagNumeric", EnParamType),
        ("IVal", ctypes.c_int),
        ("UIVal", ctypes.c_uint),
        ("FVal", ctypes.c_float),
        ("DVal", ctypes.c_double),
        ("Values", NumericParamater::PValues),
    ]

class DiagNotificationStruct(ctypes.Structure):
    _fields_ = [
        ("DiagHandle", TAsap3DiagHdl),
        ("DiagState", eServiceStates),
        ("PrivateData", ctypes.c_void_p),
    ]

class MeasurementListEntry(ctypes.Structure):
    _fields_ = [
        ("taskId", ctypes.c_ushort),
        ("rate", ctypes.c_ulong),
        ("SaveFlag", BOOL),
        ("Disabled", BOOL),
      

In [46]:
x.declarations

In [51]:
x.get_members()

In [ ]:
ctypes.Union